In [51]:

from hdimvis.data_fetchers.DataFetcher import DataFetcher
from sklearn.decomposition import PCA
from hdimvis.metrics.stress.stress import vectorised_stress
from hdimvis.metrics.distance_measures.euclidian_and_manhattan import euclidean
from hdimvis.algorithms.stochastic_ntet_algo.new_distance_calculations import compute_quartet_dhd,compute_quartet_dld
import numpy as np

In [52]:
dataset = DataFetcher.fetch_data('rna N3k')
data = dataset.data

####################
Fetching the "rna N3k" dataset
####################
Dataset loaded
Dataset shape: (3000, 50)
####################


In [94]:
Xld = PCA(n_components=2, whiten=False, copy=True).fit_transform(data).astype(np.float64)
Xld *= 10/np.std(Xld)

In [164]:
def average_batch_stress(data: np.ndarray, batch_size: int):
    N = data.shape[0]
    perms = np.arange(N)
    batch_indices =  np.arange(N - N % batch_size).reshape(-1, batch_size)
    total_stress = 0

    for batch in batch_indices:

        quartet = perms[batch]
        ld_points = Xld[quartet]
        _, Dld_quartet = compute_quartet_dld(ld_points)
        # Dld_quartet /= np.sum(Dld_quartet)
        Dhd_quartet = compute_quartet_dhd(False,data[quartet], euclidean)
        # Dhd_quartet /= np.sum(Dhd_quartet)

        stress = np.sum(( Dhd_quartet - Dld_quartet)**2)
        # stress = vectorised_stress(data[batch], np.random.randn(batch_size,2), euclidean)
        total_stress += stress

    return total_stress/batch_indices.shape[0]




In [168]:
print(average_batch_stress(data, 20))

1496092.9173862748
